# Standardized Data With Timestamps Suitable For Backtesting

### [Data Dictionary](https://www.calcbench.com/home/standardizedmetrics)
### [Documentation](http://calcbench.github.io/python_api_client/html/numeric-data.html#point-in-time)

In [ ]:
%pip install calcbench-api-client[Pandas,Backoff] tqdm

In [ ]:
import calcbench as cb
from calcbench.downloaders import iterate_and_save_pandas

## Simple Example
### Get face points only for three companies

In [ ]:
data = pd.DataFrame()
for ticker in ["MSFT", "AMZN", "XOM"]:
    company_data = cb.point_in_time(
        all_face=True,
        company_identifiers=[ticker],
        all_history=True,
        include_preliminary=True,
        include_trace=True,
    )
    data = data.append(company_data)

## Download and Save Historic Data For All Companies

In [ ]:
tickers = cb.tickers(entire_universe=True)

In [ ]:
face_output_file_name = Path.joinpath(Path.home(), "face_data.csv")

iterate_and_save_pandas(
    tickers,
    lambda ticker: cb.point_in_time(
        all_face=True,
        all_footnotes=False,
        company_identifiers=[ticker],
        all_history=True,
        include_preliminary=True,
        include_xbrl=True,
    ),
    face_output_file_name,
)

footnote_file_name = Path.joinpath(Path.home(), "footnote_data.csv")

iterate_and_save_pandas(
    tickers,
    lambda ticker: cb.point_in_time(
        all_face=False,
        all_footnotes=True,
        company_identifiers=[ticker],
        all_history=True,
        include_preliminary=False,
        include_xbrl=True,
    ),
    footnote_file_name,
)

## Push Notification of New Data
### Example @ https://github.com/calcbench/notebooks/tree/master/filing_listener

